In [2]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [4]:
basedir = './국토교통부_건축물대장_공동주택가격/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['mart_djy_08.txt']

In [11]:
f = open(basedir + filenames[0], 'r')

In [12]:
f.readline()

'11200-29959|2|집합|4|전유부|서울특별시 성동구 하왕십리동 1052번지| 서울특별시 성동구 난계로 114-31|금호 베스트빌|11200|10200|0|1052|0000||||0|112003100018|10202|0|114|31|20190101|383000000|20190528\n'

In [13]:
f.close()

In [15]:
houseprice_df = pd.read_csv(basedir + filenames[0], sep='|', header=None, encoding='euc-kr')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
print(houseprice_df.shape)
houseprice_df.head()

(113218840, 25)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,11200-29959,2,집합,4,전유부,서울특별시 성동구 하왕십리동 1052번지,서울특별시 성동구 난계로 114-31,금호 베스트빌,11200,10200,0,1052,0,NaN,NaN,NaN,0,1.120031e+11,10202.0,0,114.0,31.0,20190101,383000000,20190528
1,11200-36122,2,집합,4,전유부,서울특별시 성동구 마장동 818번지,서울특별시 성동구 살곶이길 50,마장동 현대아파트,11200,10500,0,818,0,NaN,NaN,NaN,0,1.120041e+11,10501.0,0,50.0,NaN,20190101,366000000,20190528
2,11200-73572,2,집합,4,전유부,서울특별시 성동구 옥수동 250번지,서울특별시 성동구 독서당로 218,옥수삼성아파트,11200,11300,0,250,0,NaN,NaN,NaN,0,1.120030e+11,11306.0,0,218.0,NaN,20190101,524000000,20190528
3,11110-30668,2,집합,4,전유부,서울특별시 종로구 창신동 232번지,서울특별시 종로구 지봉로5길 7,두산아파트,11110,17400,0,232,0,NaN,NaN,NaN,0,1.111041e+11,17401.0,0,7.0,NaN,20190101,390000000,20190528
4,11110-31570,2,집합,4,전유부,서울특별시 종로구 무악동 82번지,서울특별시 종로구 통일로 246-20,무악현대아파트,11110,18700,0,82,0,NaN,NaN,NaN,2,1.111030e+11,18706.0,0,246.0,20.0,20190101,614000000,20190528


In [19]:
colnames_df = pd.read_excel(basedir + '데이터구조.xls', header=1)
print(colnames_df.shape)
colnames_df.head()

(25, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [20]:
colnames = colnames_df['컬럼 한글명'].tolist()
colnames

['관리_건축물대장_PK',
 '대장_구분_코드',
 '대장_구분_코드_명',
 '대장_종류_코드',
 '대장_종류_코드_명',
 '대지_위치',
 '도로명_대지_위치',
 '건물_명',
 '시군구_코드',
 '법정동_코드',
 '대지_구분_코드',
 '번',
 '지',
 '특수지_명',
 '블록',
 '로트',
 '외필지_수',
 '새주소_도로_코드',
 '새주소_법정동_코드',
 '새주소_지상지하_코드',
 '새주소_본_번',
 '새주소_부_번',
 '기준_일자',
 '주택가격',
 '생성_일자']

In [21]:
houseprice_df.columns = colnames
houseprice_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,기준_일자,주택가격,생성_일자
0,11200-29959,2,집합,4,전유부,서울특별시 성동구 하왕십리동 1052번지,서울특별시 성동구 난계로 114-31,금호 베스트빌,11200,10200,0,1052,0,NaN,NaN,NaN,0,1.120031e+11,10202.0,0,114.0,31.0,20190101,383000000,20190528
1,11200-36122,2,집합,4,전유부,서울특별시 성동구 마장동 818번지,서울특별시 성동구 살곶이길 50,마장동 현대아파트,11200,10500,0,818,0,NaN,NaN,NaN,0,1.120041e+11,10501.0,0,50.0,NaN,20190101,366000000,20190528
2,11200-73572,2,집합,4,전유부,서울특별시 성동구 옥수동 250번지,서울특별시 성동구 독서당로 218,옥수삼성아파트,11200,11300,0,250,0,NaN,NaN,NaN,0,1.120030e+11,11306.0,0,218.0,NaN,20190101,524000000,20190528
3,11110-30668,2,집합,4,전유부,서울특별시 종로구 창신동 232번지,서울특별시 종로구 지봉로5길 7,두산아파트,11110,17400,0,232,0,NaN,NaN,NaN,0,1.111041e+11,17401.0,0,7.0,NaN,20190101,390000000,20190528
4,11110-31570,2,집합,4,전유부,서울특별시 종로구 무악동 82번지,서울특별시 종로구 통일로 246-20,무악현대아파트,11110,18700,0,82,0,NaN,NaN,NaN,2,1.111030e+11,18706.0,0,246.0,20.0,20190101,614000000,20190528


In [22]:
houseprice_df['기준_일자'].value_counts()

20180101    10189477
20170101     9803761
20160101     9442914
20150101     9112120
20200101     9081736
20190101     8938352
20140101     8861449
20130101     8408989
20120101     8380838
20110101     8179167
20100101     7833871
20090101     7510956
20080101     7171142
20100601      100132
20090601       84153
20110601       75276
20120601       44507
Name: 기준_일자, dtype: int64

In [23]:
seoul_house_price_df = houseprice_df[houseprice_df['대지_위치'].str.contains('서울')]
print(seoul_house_price_df.shape)
seoul_house_price_df.head()

(18693338, 25)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,기준_일자,주택가격,생성_일자
0,11200-29959,2,집합,4,전유부,서울특별시 성동구 하왕십리동 1052번지,서울특별시 성동구 난계로 114-31,금호 베스트빌,11200,10200,0,1052,0,NaN,NaN,NaN,0,1.120031e+11,10202.0,0,114.0,31.0,20190101,383000000,20190528
1,11200-36122,2,집합,4,전유부,서울특별시 성동구 마장동 818번지,서울특별시 성동구 살곶이길 50,마장동 현대아파트,11200,10500,0,818,0,NaN,NaN,NaN,0,1.120041e+11,10501.0,0,50.0,NaN,20190101,366000000,20190528
2,11200-73572,2,집합,4,전유부,서울특별시 성동구 옥수동 250번지,서울특별시 성동구 독서당로 218,옥수삼성아파트,11200,11300,0,250,0,NaN,NaN,NaN,0,1.120030e+11,11306.0,0,218.0,NaN,20190101,524000000,20190528
3,11110-30668,2,집합,4,전유부,서울특별시 종로구 창신동 232번지,서울특별시 종로구 지봉로5길 7,두산아파트,11110,17400,0,232,0,NaN,NaN,NaN,0,1.111041e+11,17401.0,0,7.0,NaN,20190101,390000000,20190528
4,11110-31570,2,집합,4,전유부,서울특별시 종로구 무악동 82번지,서울특별시 종로구 통일로 246-20,무악현대아파트,11110,18700,0,82,0,NaN,NaN,NaN,2,1.111030e+11,18706.0,0,246.0,20.0,20190101,614000000,20190528


In [26]:
seoul_house_price_df['기준월'] = seoul_house_price_df['기준_일자'].apply(lambda x: str(x)[4:6])

In [27]:
seoul_house_price_df['기준월'].value_counts()

01    18661342
06       31996
Name: 기준월, dtype: int64

In [28]:
seoul_house_price_df = seoul_house_price_df[seoul_house_price_df['기준월'] == '01']
print(seoul_house_price_df.shape)

(18661342, 26)


In [29]:
seoul_house_price_df.drop(columns=['기준월'], inplace=True)

In [30]:
seoul_house_price_df['기준년도'] = seoul_house_price_df['기준_일자'].apply(lambda x: int(str(x)[:4]))

In [31]:
seoul_house_price_df.drop(columns=['기준_일자', '생성_일자'], inplace=True)

In [32]:
seoul_house_price_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,주택가격,기준년도
0,11200-29959,2,집합,4,전유부,서울특별시 성동구 하왕십리동 1052번지,서울특별시 성동구 난계로 114-31,금호 베스트빌,11200,10200,0,1052,0,NaN,NaN,NaN,0,1.120031e+11,10202.0,0,114.0,31.0,383000000,2019
1,11200-36122,2,집합,4,전유부,서울특별시 성동구 마장동 818번지,서울특별시 성동구 살곶이길 50,마장동 현대아파트,11200,10500,0,818,0,NaN,NaN,NaN,0,1.120041e+11,10501.0,0,50.0,NaN,366000000,2019
2,11200-73572,2,집합,4,전유부,서울특별시 성동구 옥수동 250번지,서울특별시 성동구 독서당로 218,옥수삼성아파트,11200,11300,0,250,0,NaN,NaN,NaN,0,1.120030e+11,11306.0,0,218.0,NaN,524000000,2019
3,11110-30668,2,집합,4,전유부,서울특별시 종로구 창신동 232번지,서울특별시 종로구 지봉로5길 7,두산아파트,11110,17400,0,232,0,NaN,NaN,NaN,0,1.111041e+11,17401.0,0,7.0,NaN,390000000,2019
4,11110-31570,2,집합,4,전유부,서울특별시 종로구 무악동 82번지,서울특별시 종로구 통일로 246-20,무악현대아파트,11110,18700,0,82,0,NaN,NaN,NaN,2,1.111030e+11,18706.0,0,246.0,20.0,614000000,2019


In [33]:
simple_df = seoul_house_price_df[['대지_위치', '주택가격', '기준년도']]
simple_df.head()

,대지_위치,주택가격,기준년도
0,서울특별시 성동구 하왕십리동 1052번지,383000000,2019
1,서울특별시 성동구 마장동 818번지,366000000,2019
2,서울특별시 성동구 옥수동 250번지,524000000,2019
3,서울특별시 종로구 창신동 232번지,390000000,2019
4,서울특별시 종로구 무악동 82번지,614000000,2019


In [34]:
simple_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18661342 entries, 0 to 18693337
Data columns (total 3 columns):
대지_위치    object
주택가격     int64
기준년도     int64
dtypes: int64(2), object(1)
memory usage: 569.5+ MB


In [35]:
def addr_modifier(x):
    # x is a string
    if pd.isna(x) == True:
        return x
    else:
        x = x.replace('번지', '')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [36]:
simple_df['지번주소'] = simple_df['대지_위치'].apply(addr_modifier)
simple_df.head()

,대지_위치,주택가격,기준년도,지번주소
0,서울특별시 성동구 하왕십리동 1052번지,383000000,2019,서울특별시 성동구 하왕십리동 1052-0
1,서울특별시 성동구 마장동 818번지,366000000,2019,서울특별시 성동구 마장동 818-0
2,서울특별시 성동구 옥수동 250번지,524000000,2019,서울특별시 성동구 옥수동 250-0
3,서울특별시 종로구 창신동 232번지,390000000,2019,서울특별시 종로구 창신동 232-0
4,서울특별시 종로구 무악동 82번지,614000000,2019,서울특별시 종로구 무악동 82-0


In [37]:
simple_df = simple_df[['지번주소', '기준년도', '주택가격']]
simple_df.head()

,지번주소,기준년도,주택가격
0,서울특별시 성동구 하왕십리동 1052-0,2019,383000000
1,서울특별시 성동구 마장동 818-0,2019,366000000
2,서울특별시 성동구 옥수동 250-0,2019,524000000
3,서울특별시 종로구 창신동 232-0,2019,390000000
4,서울특별시 종로구 무악동 82-0,2019,614000000


In [38]:
simple_df = simple_df.sort_values(['지번주소', '기준년도'])

In [41]:
simple_df.reset_index(drop=True, inplace=True)

In [43]:
simple_df.head(50)

,지번주소,기준년도,주택가격
0,서울특별시 강남구 개포동 1164-12,2009,196000000
1,서울특별시 강남구 개포동 1164-12,2009,184000000
2,서울특별시 강남구 개포동 1164-12,2009,196000000
3,서울특별시 강남구 개포동 1164-12,2009,192000000
4,서울특별시 강남구 개포동 1164-12,2009,192000000
5,서울특별시 강남구 개포동 1164-12,2009,196000000
6,서울특별시 강남구 개포동 1164-12,2009,196000000
7,서울특별시 강남구 개포동 1164-12,2009,196000000
8,서울특별시 강남구 개포동 1164-12,2009,184000000
9,서울특별시 강남구 개포동 1164-12,2009,196000000


In [45]:
simple_df.drop_duplicates(subset=['지번주소', '기준년도'], inplace=True)
simple_df.head(50)

,지번주소,기준년도,주택가격
0,서울특별시 강남구 개포동 1164-12,2009,196000000
12,서울특별시 강남구 개포동 1164-12,2010,212000000
24,서울특별시 강남구 개포동 1164-12,2011,212000000
36,서울특별시 강남구 개포동 1164-12,2012,281000000
48,서울특별시 강남구 개포동 1164-12,2013,267000000
60,서울특별시 강남구 개포동 1164-12,2014,268000000
72,서울특별시 강남구 개포동 1164-12,2015,270000000
84,서울특별시 강남구 개포동 1164-12,2016,288000000
96,서울특별시 강남구 개포동 1164-12,2017,299000000
108,서울특별시 강남구 개포동 1164-12,2018,363000000


In [46]:
simple_df.reset_index(drop=True, inplace=True)
simple_df.head(50)

,지번주소,기준년도,주택가격
0,서울특별시 강남구 개포동 1164-12,2009,196000000
1,서울특별시 강남구 개포동 1164-12,2010,212000000
2,서울특별시 강남구 개포동 1164-12,2011,212000000
3,서울특별시 강남구 개포동 1164-12,2012,281000000
4,서울특별시 강남구 개포동 1164-12,2013,267000000
5,서울특별시 강남구 개포동 1164-12,2014,268000000
6,서울특별시 강남구 개포동 1164-12,2015,270000000
7,서울특별시 강남구 개포동 1164-12,2016,288000000
8,서울특별시 강남구 개포동 1164-12,2017,299000000
9,서울특별시 강남구 개포동 1164-12,2018,363000000


In [47]:
simple_df.shape

(106534, 3)

In [48]:
main_data_df = pd.read_csv('./prepped_data/main_data_ver_2.4.csv')
print(main_data_df.shape)
main_data_df.head()

(4559866, 16)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,55000,0,817.479191,5,2013,2,1987,26,10.915088,6.706225
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,58250,0,865.784780,4,2013,2,1987,26,10.972499,6.763636
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,60000,0,891.795482,5,2013,5,1987,26,11.002100,6.793237


In [49]:
simple_df.rename(columns={'기준년도':'년'}, inplace=True)

In [50]:
merge_df = main_data_df.merge(simple_df, on=['지번주소', '년'])
print(merge_df.shape)
merge_df.head()

(2402047, 17)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed,주택가격
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308,440000000
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308,440000000
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_전세,77.75,35000,0,450.160772,4,2013,1,1988,25,10.463103,6.109605,440000000
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_전세,77.75,32000,0,411.575563,9,2013,3,1988,25,10.373491,6.019993,440000000
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_전세,77.75,37000,0,475.884244,1,2013,9,1988,25,10.518673,6.165175,440000000


In [53]:
from sklearn.metrics import r2_score

In [54]:
r2_score(merge_df['주택가격'], merge_df['금액(만원)'])

-1.8141411175255873

In [62]:
main_data_df['trade_type'].value_counts()

아파트_전세      1146196
아파트_매매      1119895
다세대연립_매매     633206
다세대연립_전세     521766
아파트_월세       436729
다세대연립_월세     267774
오피스텔_매매      166109
오피스텔_월세      145667
오피스텔_전세      122524
Name: trade_type, dtype: int64

In [60]:
merge_df['trade_type'].value_counts()

아파트_전세      1054739
아파트_매매       903144
아파트_월세       391789
오피스텔_매매       20966
오피스텔_월세       17223
오피스텔_전세       13974
다세대연립_매매        122
다세대연립_전세         67
다세대연립_월세         23
Name: trade_type, dtype: int64

In [55]:
trade_only = merge_df[merge_df['trade_type'] == '아파트_매매']

In [56]:
trade_only.shape

(903144, 17)

In [57]:
r2_score(trade_only['주택가격'], trade_only['금액(만원)'])

-1.8303174989492774

In [64]:
np.corrcoef(trade_only['주택가격'], trade_only['금액(만원)'])

array([[1.        , 0.88408782],
       [0.88408782, 1.        ]])